In [1]:
# mike babb
# 2025 02 26
# simulate bingo

In [2]:
# standard
import os

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

sns.set_theme(style="ticks")


In [4]:
# the typical bingo grid is 25 squares (24 playable, 1 free play)

In [5]:
def get_array_length(arr1:np.array, arr2:np.array):
    return len(set(arr1).intersection(arr2))

In [6]:
def do_a_bingo_run(n_samples:int, n_trials:str, output_file_path:str):
    trial_list = []
    for nt in range(0, n_trials + 1):        

        # build the board
        numbers = list(range(1, n_samples + 1))
        # shuffle the numbers
        np.random.shuffle(numbers)
        # draw 24 numbers, insert 0 into the center, reshape the board to be a 5,5 matrix
        # this is the bingo board
        bb = np.insert(arr = np.array(numbers[:24]), obj = 12, values = 0).reshape(5,5)
        
        # shuffle the numbers full list of numbers again to simulate drawing
        np.random.shuffle(numbers)

        # free-play / 0 square is always already drawn
        curr_list = [0]
        keep_drawing = True 
        # start counting
        i_nn = 0
        while keep_drawing:
            nn = numbers[i_nn]
            # create a list of numbers drawn
            curr_list.append(nn)        
            # check horizontal and vertical slices
            for my_index in range(0, 5):        
                # rows        
                if get_array_length(arr1=bb[my_index, :], arr2=curr_list) >= 5:
                    trial_list.append([nt, i_nn, 0, my_index])
                    keep_drawing = False                
                # columns
                if get_array_length(arr1=bb[:, my_index], arr2=curr_list) >= 5:
                    trial_list.append([nt, i_nn, 1, my_index])
                    keep_drawing = False                
            # diagonal
            if get_array_length(arr1=bb.diagonal(), arr2=curr_list) >= 5:
                trial_list.append([nt, i_nn, 2, 1])
                keep_drawing = False
                
            # off diagonal
            if get_array_length(arr1=np.fliplr(m = bb).diagonal(), arr2=curr_list) >= 5:
                trial_list.append([nt, i_nn, 3, 2])
                keep_drawing = False
            i_nn += 1

    columns = ['trial_number', 'n_draws', 'card_part', 'idx']
    outcome = pd.DataFrame(data = trial_list, columns = columns)
    # add one to the n_draws to account for the free-space of zero
    outcome['n_draws'] = outcome['n_draws'] + 1
    outcome.shape

    output_file_name = f'bingo_sim_{str(n_trials).zfill(6)}_{str(n_samples).zfill(3)}_.csv'
    ofpn = os.path.join(output_file_path, output_file_name)
    outcome.to_csv(path_or_buf= ofpn, sep = '\t', index = False)

    return None

In [7]:
output_file_path = 'H:/project/bingo_simulator/data/'
n_trials = 10000
for ns in range(25, 51):
    print(ns)
    do_a_bingo_run(n_samples = ns, n_trials = n_trials, output_file_path = output_file_path)

25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [8]:
file_list = os.listdir(path = output_file_path)
plot_output_path = 'H:/project/bingo_simulator/print'

card_part_dict = {0:'row',
                  1:'column',
                  2:'diagonal',
                  3:'diagonal'}

In [9]:
for fn in file_list:
    print(fn)
    fpn = os.path.join(output_file_path, fn)
    df = pd.read_csv(filepath_or_buffer=fpn, sep = '\t')
    df['card_part_desc'] = df['card_part'].map(card_part_dict)
    df.head()
    # make the plot

    f, ax = plt.subplots(figsize=(20, 5))
    sns.despine(f)

    my_plot = sns.histplot(
        data = df,
        x="n_draws", hue="card_part_desc",
        multiple="stack",
        palette="Dark2",
        edgecolor=".3",
        discrete = True,
        linewidth=.5,
        cumulative=True,
        kde=True
    )

    title_parts = fn.split('_')
    n_trials = int(title_parts[2])
    n_samples = int(title_parts[3])
    my_title = f'Number of trials: {n_trials} | number of samples: {n_samples}'

    per_50 = np.quantile(a = df['n_draws'], q = .5)
    per_90 = np.quantile(a = df['n_draws'], q = .9)
    per_95 = np.quantile(a = df['n_draws'], q = .95)

    y_max = ax.get_ylim()[1] * .9

    my_plot.vlines(x = per_50, ymin = 0, ymax = y_max, color = 'red', linestyles = 'dashdot')
    my_plot.vlines(x = per_90, ymin = 0, ymax = y_max, color = 'red', linestyles = 'dashdot')
    my_plot.vlines(x = per_95, ymin = 0, ymax = y_max, color = 'red', linestyles = 'dashdot')

    plt.title(label = my_title)
    x_ticks = range(df['n_draws'].min(), df['n_draws'].max() + 1)
    ax.set_xticks(x_ticks)

    output_file_name = fn.replace('csv', 'png')
    ofpn = os.path.join(plot_output_path, output_file_name)

    my_plot.get_figure().savefig(fname = ofpn)
    plt.close()

bingo_sim_001000_025_.csv
bingo_sim_001000_026_.csv
bingo_sim_001000_027_.csv
bingo_sim_001000_028_.csv
bingo_sim_001000_029_.csv
bingo_sim_001000_030_.csv
bingo_sim_001000_031_.csv
bingo_sim_001000_032_.csv
bingo_sim_001000_033_.csv
bingo_sim_001000_034_.csv
bingo_sim_001000_035_.csv
bingo_sim_001000_036_.csv
bingo_sim_001000_037_.csv
bingo_sim_001000_038_.csv
bingo_sim_001000_039_.csv
bingo_sim_001000_040_.csv
bingo_sim_001000_041_.csv
bingo_sim_001000_042_.csv
bingo_sim_001000_043_.csv
bingo_sim_001000_044_.csv
bingo_sim_001000_045_.csv
bingo_sim_001000_046_.csv
bingo_sim_001000_047_.csv
bingo_sim_001000_048_.csv
bingo_sim_001000_049_.csv
bingo_sim_001000_050_.csv
bingo_sim_010000_025_.csv
bingo_sim_010000_026_.csv
bingo_sim_010000_027_.csv
bingo_sim_010000_028_.csv
bingo_sim_010000_029_.csv
bingo_sim_010000_030_.csv
bingo_sim_010000_031_.csv
bingo_sim_010000_032_.csv
bingo_sim_010000_033_.csv
bingo_sim_010000_034_.csv
bingo_sim_010000_035_.csv
bingo_sim_010000_036_.csv
bingo_sim_01